# `__init__` 내의 전처리
- `add_exposure_time`: 하나의 path 당 하나의 가상 노출 시간을 부여. 1은 노출시간의 간격
- `remove_loops`(optional)
- channels 라는 attribute 생성: unique한 파트너들의 sorted list  
  
다른 속성은 적어도 샤플리 계산에는 직접적으로 활용되지 않음

In [4]:
import pandas as pd
from itertools import chain, combinations
from collections import defaultdict

data = pd.read_csv("data/data.csv")
data.head()

,path,total_conversions,total_conversion_value,total_null
0,eta > iota > alpha > eta,1,0.244,3
1,iota > iota > iota > iota,2,3.195,6
2,alpha > iota > alpha > alpha > alpha > iota > ...,2,6.754,6
3,beta > eta,1,2.402,3
4,iota > eta > theta > lambda > lambda > theta >...,0,0.000,2


In [3]:
channels = sorted(list({ch for ch in chain.from_iterable(data['path'])}))
channels

[' ',
 '>',
 'a',
 'b',
 'd',
 'e',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 's',
 't',
 'z']

# 함수 `Shapely` 

## 참고 문서
https://medium.com/data-from-the-trenches/marketing-attribution-e7fa7ae9e919
Shapley model; channels are players, the characteristic function maps a coalition A to the the total number of conversions generated by all the subsets of the coalition


## 사용 함수
- 데코레이터 @showtime: 그냥 로그 찍으려고 있는 거임
- shapely에서 활용하는 클래스 함수 
    - `get_generated_conversions`: subset 별 conversion/null 합산하기
    - `normalize_dict`: (간단) 총합으로 나누는 방식으로 normalize한 리스트를 반환
    - `v`: 
    - `w`: 
- `defaultdict`: https://docs.python.org/2/library/collections.html#defaultdict-objects
- `combinations(set, n)`: set에서 크기가 n인 가능한 모든 combination 반환

## 변수
- `max_coalition_size`
- `phi`: 최종적으로 반환하는, 채널 별 기여도를 기록한 dictionary

In [ ]:
########## 주석 보라고 옮겨놓은 것. 실행 X ##########

def get_generated_conversions(self, max_subset_size=3):

    # 그냥 깡통같은 거 생성
    self.cc = defaultdict(lambda: defaultdict(float))

    # path 하나 씩 돌며 (채널들의 목록, 전환이 발생한 레코드수, 발생하지 않은 레코드수)를 같이 가져오기
    for ch_list, convs, nulls in zip(self.data['path'], 
                                        self.data['total_conversions'], 
                                            self.data['total_null']):

        # only look at journeys with conversions
        for n in range(1, max_subset_size + 1): # subset의 크기를 1부터 max까지 하나씩 늘려가며

            for tup in combinations(set(ch_list), n): # path로부터 크기가 n인 subset을 하나씩 돌기

                tup_ = self.ordered_tuple(tup) # ordered_tuple에 저장

                # cc의 key는 주어진 path로 부터 가능한 subset
                # 그에 매칭되는 cc의 value는 다시 딕셔너리인데 그 형태는
                # {'(conv)': 해당 subset을 포함하는 path가 발생시킨 conversion의 합, '(null)': 해당 subset을 포함하는 path가 발생시킨 null로 빠진 수의 합} 
                self.cc[tup_][self.CONV] += convs # 해당 subset의 '(conv)'에 현재 path의 conversion의 수를 더해주기
                self.cc[tup_][self.NULL] += nulls # 해당 subset의 '(null)'에 현재 path의 null의 수를 더해주기

    return self

## Shapley 함수 내부

In [ ]:
########## 주석 보라고 옮겨놓은 것. 실행 X ##########

@show_time
def shapley(self, max_coalition_size=2, normalize=True):

    """
    Shapley model; channels are players, the characteristic function maps a coalition A to the 
    the total number of conversions generated by all the subsets of the coalition

    see https://medium.com/data-from-the-trenches/marketing-attribution-e7fa7ae9e919
    """

    self.get_generated_conversions(max_subset_size=3) # 결과는 self.cc에 저장되어 있다

    self.phi = defaultdict(float)

    for ch in self.channels:
        # subset의 sizefmf 1부터 하나씩 키워가며
        for n in range(1, max_coalition_size + 1):
            # 현재 보고 있는 ch을 제외한 것으로 만들 수 있는, size가 n인 combination을 하나 씩 돌면서 
            for tup in combinations(set(self.channels) - {ch}, n): 
                self.phi[ch] += (self.v(tup + (ch,)) - self.v(tup))*self.w(len(tup), len(self.channels))

    if normalize:
        self.phi = self.normalize_dict(self.phi)

    self.attribution['shapley'] = self.phi

    return self

In [1]:
(1, 2) + (1,)

(1, 2, 1)

In [ ]:
np.math.factorial(s)*(np.math.factorial(n - s -1))/np.math.factorial(n)